In [ ]:
from scdesigner.datasets import pancreas as example_sce

example_sce

AnnData object with n_obs × n_vars = 2087 × 100
    obs: 'clusters_coarse', 'clusters', 'S_score', 'G2M_score', 'cell_type', 'sizeFactor', 'pseudotime'
    var: 'highly_variable_genes'
    uns: 'X_name', 'clusters_coarse_colors', 'clusters_colors', 'day_colors', 'neighbors', 'pca'
    obsm: 'PCA', 'UMAP', 'X_pca', 'X_umap'
    layers: 'counts', 'cpm', 'logcounts', 'spliced', 'unspliced'
    obsp: 'connectivities', 'distances'

Installation with a new conda environment:
```
conda create -n myenv python=3.11
conda activate myenv
pip install scdesigner==0.0.6
```

# Creating and Fitting a Simulator

Here we present an example of how to create and fit a negative binomial copula simulator.

Our simulators are designed using a `scikit-learn`-like API.

User may specify different formulas for the mean, dispersion, and copula models when initializing the simulator. Then, the simulator can be fitted to an `AnnData` object using the `fit` method.

In [ ]:
# Import simulator object from scdesigner
from scdesigner.simulators import NegBinCopula

# Create a NegBinCopula model with specified formulas
sim = NegBinCopula(mean_formula="~ pseudotime",
                   dispersion_formula="~ pseudotime",
                   copula_formula="~ -1 + cell_type")
sim.fit(example_sce, max_epochs=50)

Estimating copula covariance: 100%|██████████| 3/3 [00:00<00:00, 27.28it/s]


 There are two major components encapsulated as objects in the simulator: 
- a marginal model `sim.marginal`, and 
- a copula model `sim.copula`.

To access the parameters of the marginal model:

In [4]:
display(sim.marginal.parameters['mean'])
display(sim.marginal.parameters['dispersion'])

,Pyy,Iapp,Chgb,Rbp4,Spp1,Chga,Cck,Ins1,Nnat,Ins2,...,Nkx6-1,Fxyd3,Hn1,Smarcd2,Pdia6,Ffar2,Hes6,Serpinh1,Npy,1110012L19Rik
Intercept,0.117757,0.117602,0.119631,0.118169,0.120878,0.119587,0.119796,0.117452,0.118364,0.117997,...,0.118660,0.114883,0.118461,0.116519,0.117698,0.116810,0.118764,0.117988,-0.025031,0.111965
pseudotime,0.117791,0.117915,0.119517,0.118309,0.119103,0.119552,0.119612,0.117586,0.118557,0.118351,...,0.118424,0.115241,0.116935,0.108638,0.117538,0.116978,0.116029,0.086943,0.115997,0.107555


,Pyy,Iapp,Chgb,Rbp4,Spp1,Chga,Cck,Ins1,Nnat,Ins2,...,Nkx6-1,Fxyd3,Hn1,Smarcd2,Pdia6,Ffar2,Hes6,Serpinh1,Npy,1110012L19Rik
Intercept,-0.117807,-0.117818,-0.119528,-0.118012,-0.121260,-0.119416,-0.119880,-0.117756,-0.118612,-0.118911,...,-0.107236,-0.091895,-0.111850,-0.105489,-0.111612,-0.085886,-0.110246,-0.110723,-0.122163,-0.111293
pseudotime,-0.117610,-0.117939,-0.119349,-0.117905,-0.123616,-0.118831,-0.119446,-0.117777,-0.118588,-0.118779,...,-0.107952,-0.089361,-0.099151,-0.110213,-0.113651,-0.075710,-0.081252,-0.106253,-0.120922,-0.112743


Since the copula model is fitted on `cell_type`, each cell type has its own fitted covariance matrix.

The parameters are stored as a dictionary of covariance matrices, one for each cell type.

In [5]:
# Print the cell types (keys) in the copula model
print(sim.copula.parameters.keys())

dict_keys(['cell_type[T.Ngn3 low EP]', 'cell_type[T.Ngn3 high EP]', 'cell_type[T.Pre-endocrine]', 'cell_type[T.Beta]'])


In [6]:
# Show the covariance matrix for a specific cell type
sim.copula.parameters['cell_type[T.Ngn3 low EP]']

,Pyy,Iapp,Chgb,Rbp4,Spp1,Chga,Cck,Ins1,Nnat,Ins2,...,Nkx6-1,Fxyd3,Hn1,Smarcd2,Pdia6,Ffar2,Hes6,Serpinh1,Npy,1110012L19Rik
Pyy,0.771580,0.038014,-0.003788,0.086433,-0.009579,-0.058163,0.055087,0.041658,0.028645,-0.015799,...,-0.021059,0.016673,0.043605,0.032012,-0.052223,-0.040791,0.049224,-0.008445,-0.020720,0.056972
Iapp,0.038014,0.505641,0.012560,0.030906,-0.019210,0.027883,0.084122,0.017225,-0.057427,-0.029827,...,-0.033980,0.024461,0.053416,0.030284,-0.000984,0.015464,0.077967,-0.018177,0.035081,-0.016420
Chgb,-0.003788,0.012560,0.443511,0.016112,0.006366,0.021101,-0.043805,0.014282,-0.037524,0.007086,...,0.004307,-0.012655,0.060461,-0.030653,-0.024514,-0.053281,0.020575,-0.010844,-0.037114,-0.013198
Rbp4,0.086433,0.030906,0.016112,0.789852,0.014855,-0.035903,0.025809,0.042682,0.015363,-0.011954,...,0.057966,-0.010574,0.109575,-0.028280,0.084860,-0.007925,-0.005315,0.043961,-0.009444,-0.028888
Spp1,-0.009579,-0.019210,0.006366,0.014855,0.140114,-0.005783,-0.048087,0.003335,-0.012987,-0.010404,...,0.026080,-0.003717,-0.059614,-0.034746,0.023644,0.001383,-0.027112,0.046331,-0.021179,-0.013223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ffar2,-0.040791,0.015464,-0.053281,-0.007925,0.001383,0.062829,0.002575,0.019052,-0.072236,-0.031974,...,-0.030911,0.063905,0.121044,0.096398,0.011081,0.765600,0.105765,0.009385,-0.031113,-0.059821
Hes6,0.049224,0.077967,0.020575,-0.005315,-0.027112,0.043232,0.049773,-0.039740,-0.050994,-0.061792,...,0.011850,0.074063,0.347564,0.101468,0.024565,0.105765,0.968072,0.041418,-0.087185,0.019077
Serpinh1,-0.008445,-0.018177,-0.010844,0.043961,0.046331,-0.077327,0.018311,0.006388,0.039746,0.046168,...,-0.019317,0.042216,0.042160,0.075796,0.198619,0.009385,0.041418,0.724652,-0.044406,0.043589
Npy,-0.020720,0.035081,-0.037114,-0.009444,-0.021179,0.000692,0.008599,0.048602,0.027321,-0.052117,...,-0.009678,0.020747,-0.015421,-0.052759,0.037460,-0.031113,-0.087185,-0.044406,0.360296,-0.051039


# Predicting and Sampling from the Simulator

You can predict and sample from the simulator using the `predict` and `sample` methods and passing in the covariates of interest.

You may also obtain the aic and bic scores of the fitted model using the `complexity` method.

In [7]:
# Predict the mean and dispersion based on given covariates
preds = sim.predict(example_sce.obs[:10])
pd.DataFrame(preds['mean'])

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.218388,1.218302,1.222103,1.219319,1.223284,1.222077,1.222383,1.217848,1.219762,1.219144,...,1.220013,1.212796,1.218540,1.209362,1.218109,1.216566,1.218162,1.193474,1.054984,1.202983
1,1.183637,1.183517,1.186742,1.184389,1.188009,1.186706,1.186986,1.183171,1.184748,1.184208,...,1.185030,1.178942,1.184033,1.177512,1.183438,1.182102,1.183929,1.168253,1.025345,1.171614
2,1.228785,1.228709,1.232685,1.229769,1.233839,1.232662,1.232976,1.228222,1.230238,1.229597,...,1.230480,1.222920,1.228862,1.218876,1.228481,1.226875,1.228401,1.200983,1.063849,1.212353
3,1.254535,1.254485,1.258899,1.255655,1.259986,1.258883,1.259217,1.253915,1.256188,1.255488,...,1.256405,1.247987,1.254424,1.242415,1.254168,1.252406,1.253754,1.219509,1.085800,1.235530
4,1.176516,1.176390,1.179498,1.177233,1.180783,1.179461,1.179736,1.176066,1.177575,1.177050,...,1.177863,1.172003,1.176962,1.170978,1.176334,1.175040,1.176913,1.163062,1.019271,1.165177
5,1.159454,1.159312,1.162144,1.160086,1.163470,1.162103,1.162364,1.159040,1.160387,1.159900,...,1.160691,1.155372,1.160016,1.155306,1.159311,1.158116,1.160099,1.150588,1.004713,1.149738
6,1.128872,1.128702,1.131048,1.129355,1.132445,1.130999,1.131238,1.128521,1.129584,1.129162,...,1.129914,1.125548,1.129639,1.127173,1.128798,1.127777,1.129951,1.128110,0.978611,1.122015
7,1.219996,1.219912,1.223739,1.220935,1.224916,1.223714,1.224021,1.219452,1.221382,1.220761,...,1.221632,1.214362,1.220136,1.210834,1.219713,1.218161,1.219746,1.194637,1.056356,1.204433
8,1.241839,1.241776,1.245973,1.242891,1.247093,1.245953,1.246277,1.241246,1.243393,1.242721,...,1.243622,1.235629,1.241821,1.230813,1.241503,1.239818,1.241254,1.210387,1.074977,1.224108
9,1.212805,1.212713,1.216420,1.213706,1.217615,1.216393,1.216694,1.212276,1.214135,1.213530,...,1.214391,1.207358,1.212996,1.204249,1.212538,1.211029,1.212663,1.189435,1.050223,1.197948


In [ ]:
# Generating new samples from the simulator
samples = sim.sample(example_sce.obs[:10])
samples

AnnData object with n_obs × n_vars = 10 × 100
    obs: 'clusters_coarse', 'clusters', 'S_score', 'G2M_score', 'cell_type', 'sizeFactor', 'pseudotime'

In [9]:
sim.complexity()

Computing log-likelihood...: 100%|██████████| 3/3 [00:00<00:00, 22.99it/s]


{'aic': 1051118.8266461124, 'bic': 1162859.7882062409}

# Manipulating the Simulator

scDesigner provides a set of tools to manipulate the simulator.

For example, we can decorrelate certain genes in the copula model.


In [10]:
sim.copula.parameters['cell_type[T.Ngn3 low EP]']

,Pyy,Iapp,Chgb,Rbp4,Spp1,Chga,Cck,Ins1,Nnat,Ins2,...,Nkx6-1,Fxyd3,Hn1,Smarcd2,Pdia6,Ffar2,Hes6,Serpinh1,Npy,1110012L19Rik
Pyy,0.771580,0.038014,-0.003788,0.086433,-0.009579,-0.058163,0.055087,0.041658,0.028645,-0.015799,...,-0.021059,0.016673,0.043605,0.032012,-0.052223,-0.040791,0.049224,-0.008445,-0.020720,0.056972
Iapp,0.038014,0.505641,0.012560,0.030906,-0.019210,0.027883,0.084122,0.017225,-0.057427,-0.029827,...,-0.033980,0.024461,0.053416,0.030284,-0.000984,0.015464,0.077967,-0.018177,0.035081,-0.016420
Chgb,-0.003788,0.012560,0.443511,0.016112,0.006366,0.021101,-0.043805,0.014282,-0.037524,0.007086,...,0.004307,-0.012655,0.060461,-0.030653,-0.024514,-0.053281,0.020575,-0.010844,-0.037114,-0.013198
Rbp4,0.086433,0.030906,0.016112,0.789852,0.014855,-0.035903,0.025809,0.042682,0.015363,-0.011954,...,0.057966,-0.010574,0.109575,-0.028280,0.084860,-0.007925,-0.005315,0.043961,-0.009444,-0.028888
Spp1,-0.009579,-0.019210,0.006366,0.014855,0.140114,-0.005783,-0.048087,0.003335,-0.012987,-0.010404,...,0.026080,-0.003717,-0.059614,-0.034746,0.023644,0.001383,-0.027112,0.046331,-0.021179,-0.013223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ffar2,-0.040791,0.015464,-0.053281,-0.007925,0.001383,0.062829,0.002575,0.019052,-0.072236,-0.031974,...,-0.030911,0.063905,0.121044,0.096398,0.011081,0.765600,0.105765,0.009385,-0.031113,-0.059821
Hes6,0.049224,0.077967,0.020575,-0.005315,-0.027112,0.043232,0.049773,-0.039740,-0.050994,-0.061792,...,0.011850,0.074063,0.347564,0.101468,0.024565,0.105765,0.968072,0.041418,-0.087185,0.019077
Serpinh1,-0.008445,-0.018177,-0.010844,0.043961,0.046331,-0.077327,0.018311,0.006388,0.039746,0.046168,...,-0.019317,0.042216,0.042160,0.075796,0.198619,0.009385,0.041418,0.724652,-0.044406,0.043589
Npy,-0.020720,0.035081,-0.037114,-0.009444,-0.021179,0.000692,0.008599,0.048602,0.027321,-0.052117,...,-0.009678,0.020747,-0.015421,-0.052759,0.037460,-0.031113,-0.087185,-0.044406,0.360296,-0.051039


Decorrelate `Pyy` and `Iapp`

In [11]:
sim.copula.decorrelate("Pyy|Iapp", "Pyy", 'cell_type[T.Ngn3 low EP]')
sim.copula.parameters['cell_type[T.Ngn3 low EP]']

,Pyy,Iapp,Chgb,Rbp4,Spp1,Chga,Cck,Ins1,Nnat,Ins2,...,Nkx6-1,Fxyd3,Hn1,Smarcd2,Pdia6,Ffar2,Hes6,Serpinh1,Npy,1110012L19Rik
Pyy,0.77158,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Iapp,0.00000,0.505641,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Chgb,0.00000,0.012560,0.443511,0.016112,0.006366,0.021101,-0.043805,0.014282,-0.037524,0.007086,...,0.004307,-0.012655,0.060461,-0.030653,-0.024514,-0.053281,0.020575,-0.010844,-0.037114,-0.013198
Rbp4,0.00000,0.030906,0.016112,0.789852,0.014855,-0.035903,0.025809,0.042682,0.015363,-0.011954,...,0.057966,-0.010574,0.109575,-0.028280,0.084860,-0.007925,-0.005315,0.043961,-0.009444,-0.028888
Spp1,0.00000,-0.019210,0.006366,0.014855,0.140114,-0.005783,-0.048087,0.003335,-0.012987,-0.010404,...,0.026080,-0.003717,-0.059614,-0.034746,0.023644,0.001383,-0.027112,0.046331,-0.021179,-0.013223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ffar2,0.00000,0.015464,-0.053281,-0.007925,0.001383,0.062829,0.002575,0.019052,-0.072236,-0.031974,...,-0.030911,0.063905,0.121044,0.096398,0.011081,0.765600,0.105765,0.009385,-0.031113,-0.059821
Hes6,0.00000,0.077967,0.020575,-0.005315,-0.027112,0.043232,0.049773,-0.039740,-0.050994,-0.061792,...,0.011850,0.074063,0.347564,0.101468,0.024565,0.105765,0.968072,0.041418,-0.087185,0.019077
Serpinh1,0.00000,-0.018177,-0.010844,0.043961,0.046331,-0.077327,0.018311,0.006388,0.039746,0.046168,...,-0.019317,0.042216,0.042160,0.075796,0.198619,0.009385,0.041418,0.724652,-0.044406,0.043589
Npy,0.00000,0.035081,-0.037114,-0.009444,-0.021179,0.000692,0.008599,0.048602,0.027321,-0.052117,...,-0.009678,0.020747,-0.015421,-0.052759,0.037460,-0.031113,-0.087185,-0.044406,0.360296,-0.051039


# Documentation Plan

We are working on numpy-style docstrings for the scDesigner package, with simple examples included


In [100]:
from scdesigner.base.copula import Copula
print(Copula.__doc__)

Abstract Copula Class
    
    The scDesign3 model is built from two components: a collection of marginal
    models, and a copula to tie them together. This class implements an abstract
    version of the copula. Within this class, we may define different subclasses
    that implement various types of regularization or dependencies on
    experimental and biological conditions. Despite these differences, the
    overall class must always provide utilities for fitting and sampling
    dependent uniform variables.
    
    Parameters
    ----------
    formula : str
        A string describing the dependence of the copula on experimental or
        biological conditions. We support predictors for categorical variables
        like cell type; this corresponds to estimating a different covariance
        for each category.
    Attributes
    ----------
    loader : torch.utils.data.DataLoader
        A data loader object is used to estimate the covariance one batch at a
        time. This

In [101]:
print(Copula.decorrelate.__doc__)


        Decorrelate the covariance matrix for the given row and column patterns.
        This method can be used to generate synthetic null data where particular
        pairs of features are forced to be uncorrelated with one another. Any
        indices of the covariance that lie in the intersection of the specified
        row and column patterns will be set to zero.
        
        Parameters
        ----------
        row_pattern : str
            The regex pattern for the row names to match.
        col_pattern : str
            The regex pattern for the column names to match.
        group : Union[str, list, None], optional
            The group or groups to apply the transformation to. If None, the
            transformation is applied to all groups.
            
        Returns
        -------
        None
            This method does not return anything but modifies self parameters as
            a side effect.
        
